In [13]:
import os
import re
import pandas as pd
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging
from tqdm import tqdm
import torch

In [14]:
def parse_hotel_reviews(data_dir):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    # Updated regex pattern to handle flexible spacing and separators
    date_pattern = re.compile(r'(\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\s\d{4})\s+([^\t\n]*)\s+(.*)')
    data_frames = []

    for root, dirs, files in os.walk(data_dir):
        city = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            hotel_name = os.path.splitext(file)[0]  # Assuming the file has no extension
            try:
                with open(file_path, 'r', encoding='ISO-8859-1') as f:
                    content = f.read()

                if not content:
                    logging.warning(f'File {file_path} is empty.')
                    continue

                reviews = date_pattern.findall(content)
                if not reviews:
                    logging.warning(f'No reviews found in file {file_path}. Content: {content[:500]}')
                    continue

                reviews_data = [{'hotel_name': hotel_name, 'hotel_city': city, 'review_date': date, 'hotel_review': review.strip()} for date, _, title, review in reviews]
                if reviews_data:
                    data_frames.append(pd.DataFrame(reviews_data))

            except Exception as e:
                logging.error(f'Error processing file {file_path}: {e}')
                continue

    if data_frames:
        review_df = pd.concat(data_frames, ignore_index=True)
    else:
        review_df = pd.DataFrame(columns=['hotel_name', 'hotel_city', 'review_date', 'hotel_review'])

    return review_df

dataset = 'beijing_roberta'
data_dir = 'data'
review_df = parse_hotel_reviews(data_dir)
print(review_df.head())
review_df.to_csv(f'csv/{dataset}_derlenmis_reviews.csv', index=False)

                                          hotel_name hotel_city  review_date  \
0  uk_england_london_130_queensgate_london_apartm...     london  Aug 14 2007   
1  uk_england_london_130_queensgate_london_apartm...     london  Oct 17 2006   
2  uk_england_london_130_queensgate_london_apartm...     london   Aug 9 2006   
3  uk_england_london_130_queensgate_london_apartm...     london   Aug 8 2006   
4  uk_england_london_130_queensgate_london_apartm...     london  Jun 12 2006   

                                        hotel_review  
0  I've stayed at the Queensgate on several occas...  
1  We stayed at 130 Queensgate Apartments for 6 d...  
2  Having read the reviews on this site, I though...  
3  This is a foul hotel. From the dingy, peeling ...  
4  130 Queensgate is, as others state, a bit drea...  


In [15]:
review_df = pd.read_csv(f'csv/{dataset}_derlenmis_reviews.csv')
review_df = review_df.dropna(subset=['hotel_review'])
review_df['processed_review'] = review_df['hotel_review'].str.lower().str.replace('\t', ' ', regex=False)
review_df = review_df.drop('hotel_review', axis=1)
review_df = review_df.drop('review_date', axis=1)
review_df.to_csv(f'csv/{dataset}_processed_reviews.csv', index=False)

In [16]:
roberta_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
roberta_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
roberta_pipeline = pipeline("sentiment-analysis", model=roberta_model, tokenizer=roberta_tokenizer)

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
aspects = ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']
weights = {'LABEL_0': -1, 'LABEL_1': 0, 'LABEL_2': 1}
batch_size = 16

aspect_keywords = {
    'cleanliness': ['clean', 'dirty', 'smell', 'stink', 'stunk', 'filthy'],
    'room': ['room', 'bed', 'suite', 'large'],
    'service': ['service', 'staff', 'help', 'support'],
    'location': ['location', 'close', 'area', 'far'],
    'value': ['value', 'worth', 'price'],
    'safety': ['safe', 'safety', 'secure', 'danger', 'dangerous'],
    'comfort': ['comfort', 'comfortable', 'uncomfortable'],
    'transportation': ['bus', 'metro', 'station', 'close', 'walk'],
    'noise': ['sound', 'volume', 'noisy', 'noise']
}

def is_aspect_mentioned(review, aspect):
    keywords = aspect_keywords[aspect]
    return any(keyword in review for keyword in keywords)

for aspect in aspects:
    review_df[f'{aspect}_score'] = 0

review_df = pd.read_csv(f'csv/{dataset}_processed_reviews.csv')

In [18]:
def process_reviews(pipeline, weights):
    for aspect in aspects:
        review_df[f'{aspect}_score'] = 0

    for aspect in aspects:
        print(f"Starting processing for aspect: {aspect}")
        for i in tqdm(range(0, len(review_df), batch_size), desc=f"Batches for {aspect}"):
            batch_reviews = review_df['processed_review'][i:i + batch_size]
            batch_index = batch_reviews.index
            aspect_mentioned = [is_aspect_mentioned(review, aspect) for review in batch_reviews]
            if any(aspect_mentioned):
                filtered_reviews = [review for review, mentioned in zip(batch_reviews, aspect_mentioned) if mentioned]
                aspect_reviews = [f"{aspect}: {review[:512 - len(aspect) - 2]}" for review in filtered_reviews]
                results = pipeline(aspect_reviews)
                scores = [weights[result['label']] * result['score'] for result in results]
                score_index = [index for index, mentioned in zip(batch_index, aspect_mentioned) if mentioned]
                review_df.loc[score_index, f'{aspect}_score'] = scores
            else:
                review_df.loc[batch_index, f'{aspect}_score'] = 0
    for aspect in aspects:
        review_df[f'{aspect}_score'] = pd.to_numeric(review_df[f'{aspect}_score'], errors='coerce')

    return review_df

In [19]:
review_df = process_reviews(roberta_pipeline, weights)

Starting processing for aspect: cleanliness


Batches for cleanliness:   0%|          | 0/79 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_11500\3139876838.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.0, -0.9531195163726807, -0.529599666595459, -0.8528810739517212, 0.9093777537345886, 0.8807716965675354, 0.9830580353736877]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for cleanliness: 100%|██████████| 79/79 [01:48<00:00,  1.37s/it]


Finished processing for aspect: cleanliness
Starting processing for aspect: room


Batches for room:   0%|          | 0/79 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_11500\3139876838.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.0, 0.0, 0.9074041843414307, -0.9621181488037109, 0.5138640999794006, -0.5216992497444153, -0.5794516205787659, 0.657778799533844, -0.9412140846252441, -0.9699875116348267, 0.9551610946655273, -0.8503766059875488, 0.9166836738586426, 0.9125731587409973, -0.8508493304252625, 0.9831101894378662]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for room: 100%|██████████| 79/79 [02:42<00:00,  2.06s/it]


Finished processing for aspect: room
Starting processing for aspect: service


...

Finished processing for aspect: comfort
Starting processing for aspect: transportation


Batches for transportation:   0%|          | 0/79 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_11500\3139876838.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.0, -0.40702295303344727, 0.8975889682769775, -0.9592729806900024, 0.44931718707084656, -0.5139015316963196, 0.6443576216697693, -0.8365989923477173, 0.851690948009491, 0.983511209487915]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for transportation: 100%|██████████| 79/79 [01:43<00:00,  1.31s/it]


Finished processing for aspect: transportation
Starting processing for aspect: noise


Batches for noise:   0%|          | 0/79 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_11500\3139876838.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.41632601618766785, 0.9095656871795654, 0.5094438791275024, -0.5160045027732849, -0.617023766040802]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for noise: 100%|██████████| 79/79 [00:32<00:00,  2.46it/s]

Finished processing for aspect: noise


In [26]:
review_df.to_csv(f'csv/{dataset}_final_reviewdf.csv', index=False)

In [23]:
aggregated_scores = review_df.groupby(['hotel_name', 'hotel_city']).agg({
    'cleanliness_score': 'mean',
    'room_score': 'mean',
    'service_score': 'mean',
    'location_score': 'mean',
    'value_score': 'mean',
    'safety_score': 'mean',
    'comfort_score': 'mean',
    'transportation_score': 'mean',
    'noise_score': 'mean'
}).reset_index()

aggregated_scores.to_csv(f'csv/{dataset}_aggregated_hotel_scores.csv', index=False)

In [24]:
display(aggregated_scores)

,hotel_name,hotel_city,cleanliness_score,room_score,service_score,location_score,value_score,safety_score,comfort_score,transportation_score,noise_score
0,uk_england_london_130_queensgate_london_apartm...,london,0.020364,0.193686,0.124379,0.152285,0.348658,-0.031596,-0.020053,0.136241,-0.004828
1,uk_england_london_190_bermondsey_street,london,0.222542,0.217091,0.203607,0.209443,0.467709,0.000000,0.220552,0.467844,0.467806
2,uk_england_london_196_bishopsgate,london,0.329965,0.548026,0.326695,0.462811,0.235546,0.025378,0.206014,0.581035,0.298362
3,uk_england_london_1_lexham_gardens,london,0.101656,0.029257,0.132357,0.116306,0.137201,-0.021890,0.002325,0.104823,0.019873
4,uk_england_london_22_jermyn_street,london,0.155326,0.682698,0.691145,0.638680,0.262500,0.000000,0.339774,0.346505,0.293957
5,uk_england_london_39_suites,london,0.177335,0.092709,-0.107901,0.278349,0.101045,0.078264,0.092041,0.223019,0.023736
6,uk_england_london_44_curzon_street_apartments,london,0.133048,0.220923,0.452429,0.408535,0.108074,0.233598,0.296918,0.170394,-0.013268
7,uk_england_london_51_buckingham_gate_taj_suite...,london,0.192068,0.607417,0.702693,0.608224,0.174977,0.049547,0.145319,0.417071,0.044997
8,uk_england_london_51_kensington_court_limited,london,0.350148,0.347039,0.580541,0.653625,0.227542,0.000000,0.323267,0.539355,0.082231
9,uk_england_london_54_boutique_hotel,london,0.236312,0.511619,0.428879,0.447458,0.184628,0.072152,0.189109,0.288211,0.123839
